In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from datetime import datetime
from difflib import SequenceMatcher
import numpy as np
from fuzzywuzzy import fuzz

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
# Define the file path relative to the current notebook's location
file_path = 'complaints.csv' 
# Read the CSV file into a DataFrame
cfpb_df = pd.read_csv(file_path)

/Users/justina-irene/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3135: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
cfpb_df.sample(5)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
1932207,2022-01-09,Debt collection,Credit card debt,Written notification about debt,Didn't receive enough information to verify debt,Did not get the summons to go to court for thi...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",AL,35244,NaN,Consent provided,Web,2022-01-09,Closed with explanation,Yes,NaN,5087593
1399688,2015-08-28,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,TOYOTA FINANCIAL IS REQUESTING PAYMENTS THROUG...,NaN,TOYOTA MOTOR CREDIT CORPORATION,PR,00729,NaN,Consent provided,Web,2015-09-02,Closed with explanation,Yes,No,1541547
1876059,2022-07-01,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,I was in the process of applying for new emplo...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,AZ,85006.0,NaN,Consent provided,Web,2022-07-01,Closed with explanation,Yes,NaN,5724807
431629,2021-11-07,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,WI,53209,NaN,Consent not provided,Web,2021-11-07,Closed with non-monetary relief,Yes,NaN,4885269
3017417,2017-01-26,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,CAPITAL ONE FINANCIAL CORPORATION,MA,20785,Older American,Consent not provided,Web,2017-01-31,Closed with explanation,Yes,No,2313324


In [6]:
%%time
# Convert the 'date' column to datetime format
print("Before droping nan narrative: ", len(cfpb_df))
cfpb_df.dropna(subset=['Consumer complaint narrative'], inplace=True)
cfpb_df['Date received'] = pd.to_datetime(cfpb_df['Date received'])
cfpb_df['narr_len'] = cfpb_df['Consumer complaint narrative'].apply(lambda x:len(str(x)))
cfpb_df['days_to_today'] = (datetime.now().date() - cfpb_df['Date received'].dt.date).dt.days

cfpb_df['narr_len'] = cfpb_df['narr_len'].astype(int)
cfpb_df['days_to_today'] = cfpb_df['days_to_today'].astype(int)


cfpb_df['narr_head'] = cfpb_df['Consumer complaint narrative'].apply(lambda x: x[:200])
print("After droping nan narrative: ", len(cfpb_df))

Before droping nan narrative:  3538637
After droping nan narrative:  1275581
CPU times: user 4.68 s, sys: 357 ms, total: 5.04 s
Wall time: 5.06 s


In [7]:
small_cfpb_df = cfpb_df[['Product', 'Issue', 'State', 'ZIP code','Complaint ID','narr_len', 'days_to_today', 'narr_head']].copy()
small_cfpb_df[['Product', 'Issue', 'State', 'ZIP code']] = small_cfpb_df[['Product', 'Issue', 'State', 'ZIP code']].fillna('')

In [8]:
# when running group by, the input is a mini dataframe with same ['Product', 'Issue', 'State', 'ZIP code']
def find_duplicate_narr(df):
    small_df = df[['Complaint ID','narr_len', 'days_to_today', 'narr_head']].copy()
    
    def find_dupi_in_small_df(row_narr_len, row_to_day, row_short_narr, small_df):
#         tmp_df = small_df.query("'narr_len'<=%s & 'narr_len'>=%s & 'days_to_today'<= %s & 'days_to_today'>= %s" % (int(row_narr_len*1.2),
#                                                                                                                    int(row_narr_len*0.8),
#                                                                                                                    int(row_to_day+15),
#                                                                                                                    int(row_to_day-15))).copy()
        tmp_df = small_df.query("narr_len <= @row_narr_len*1.2 & narr_len >= @row_narr_len*0.8 & days_to_today <= @row_to_day+15 & days_to_today >= @row_to_day-15").copy()    
        # tmp_df['similarity_to_row'] = tmp_df['narr_head'].apply(lambda x: SequenceMatcher(None, x, row_short_narr).ratio())
        tmp_df['similarity_to_row'] = tmp_df['narr_head'].apply(lambda x: fuzz.WRatio(x, row_short_narr)/100)
        
        dupli_df = tmp_df[tmp_df['similarity_to_row']>0.85]
        dupli_id_list = sorted(dupli_df['Complaint ID'].to_list())
#         if len(dupli_id_list)>1:
#             print(dupli_id_list)
        return dupli_id_list
    
    df['dupi_id'] = small_df.apply(lambda row: find_dupi_in_small_df(row['narr_len'], int(row['days_to_today']), row['narr_head'], small_df),axis=1)
    
    return df

In [9]:
%%time
small_cfpb_df = small_cfpb_df.groupby(['Product', 'Issue', 'State', 'ZIP code']).apply(func=find_duplicate_narr)

CPU times: user 1h 34min 55s, sys: 49.1 s, total: 1h 35min 44s
Wall time: 1h 36min 15s


In [10]:
small_cfpb_df['dupi_len'] = small_cfpb_df['dupi_id'].apply(lambda x: len(x))
small_cfpb_df['dupi_id'] = small_cfpb_df['dupi_id'].apply(lambda x: ";".join([str(y) for y in x]))

In [11]:
merged_df = cfpb_df.merge(small_cfpb_df[['Complaint ID', 'dupi_id', 'dupi_len']], on='Complaint ID', how='left').drop(['narr_head'], axis=1)

In [13]:
merged_df.to_csv('CFPB with Duplicate Marked.csv')
merged_df.to_csv('CFPB with Duplicate Marked fuzz.csv')

In [14]:
merged_df[merged_df.dupi_len>1]

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,narr_len,days_to_today,dupi_id,dupi_len
10,2022-03-02,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,"Regardless of the multiple written requests, t...",NaN,"EQUIFAX, INC.",NY,11219.0,...,Web,2022-03-02,Closed with explanation,Yes,NaN,5279068,549,437,5279064;5279068;5279788;5284608;5284615;528461...,8
16,2022-03-02,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I had my personal information stolen and i am ...,NaN,"EQUIFAX, INC.",TX,77373.0,...,Web,2022-03-02,Closed with explanation,Yes,NaN,5276582,328,437,5276582;5276583;5280911,3
26,2023-02-19,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,I sent a letter off on XX/XX/2023. To remove p...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AE,9104.0,...,Web,2023-02-19,Closed with non-monetary relief,Yes,NaN,6591660,382,83,6591660;6591661,2
27,2023-02-16,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I AM SUBMITTING THIS WITHOUT ANY INFLUENCE AND...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",SC,29560.0,...,Web,2023-02-16,Closed with non-monetary relief,Yes,NaN,6574595,534,86,6531393;6574595,2
31,2023-02-08,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,"According to my research about FCRA 605, credi...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,GA,30013.0,...,Web,2023-02-08,Closed with explanation,Yes,NaN,6549135,833,94,6549125;6549126;6549135,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275488,2015-12-04,Credit reporting,NaN,Incorrect information on credit report,Personal information,"I have never been In Kansas, Oklahoma ; I was ...",NaN,"EQUIFAX, INC.",TX,75083.0,...,Web,2015-12-04,Closed with explanation,Yes,No,1682881,127,2717,1677253;1682881,2
1275489,2015-03-19,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,"Identity was stolen, and inaccurate informatio...",NaN,"EQUIFAX, INC.",SC,29045.0,...,Web,2015-03-23,Closed with explanation,Yes,No,1291223,86,2977,1291222;1291223,2
1275497,2015-08-17,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,My credit report includes my parents information.,NaN,"EQUIFAX, INC.",PA,19087.0,...,Web,2015-08-17,Closed with explanation,Yes,No,1521227,49,2826,1521225;1521226;1521227,3
1275567,2022-04-14,Debt collection,Credit card debt,Attempts to collect debt not owed,Debt was paid,"Dear CFPB Team, The reason for my complaint is...",NaN,PORTFOLIO RECOVERY ASSOCIATES INC,NY,10302.0,...,Web,2022-04-14,Closed with explanation,Yes,NaN,5447900,4586,394,5447900;5460193,2


In [15]:
merged_df.shape

(1275581, 22)

In [ ]:
349816/1300361